This notebook contains processing basics and visualizations to help you understand different NLP techniques that you have studied. We will illustrate the steps mainly using built-in nltk and spacy functions. They are purely meant for presentation and understanding. We do not expect you to implement those things as part of any requirement.

## Part 1: Lexical analysis

The first part of the notebook covers lexical analysis step on the COVID tweet dataset, specifically, part-of-speech (POS) tagging. 

### Applying POS tagging to COVID tweet dataset

In [48]:
# Import libraries here that you need for different processing steps
import nltk
import csv
import spacy
import pandas as pd

In [49]:
# Read csv into dataframe and remove lines which contain missing values in the OriginalTweet column

data_df = pd.read_csv("Dataset/covid.csv")

print ("Data set: ", len(data_df))

data_df = data_df[data_df['OriginalTweet'].notna()]
print ("Data set: ", len(data_df))

data_df.head()

### Functions for Penn Treebank-style tokenization and POS tagging

#### POS tagging using nltk
Refer to this link for explanation of all POS tag abbreviations: 
https://www.guru99.com/pos-tagging-chunking-nltk.html

In [35]:
import nltk

# class for tokenization
class Splitter(object):
    # load the tokenizer
    def __init__(self):
        self.nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()
    #split input 
    def split(self, text):
        sentences = self.nltk_splitter.tokenize(text)
        tokenized_sentences = [self.nltk_tokenizer.tokenize(sent) for sent in sentences]
        return tokenized_sentences

# class for POS tagging
class POSTagger(object):
    def __init__(self):
        pass
    def pos_tag(self, sentences):
        pos = [nltk.pos_tag(sentence) for sentence in sentences]
        pos = [[(word, word, [postag]) for (word, postag) in sentence] for sentence in pos]
        return pos
    
splitter = Splitter()
postagger = POSTagger()

In [50]:
# POS tagging on an example tweet 
# We use OriginalTweet as input because stemming and stopword removal would make POS tagging somewhat meaningless, as the integrity of sentences/tokens is violated. 

print(data_df.OriginalTweet.tolist()[98])
print("\n")

tweet = data_df.OriginalTweet.tolist()[98]
splitted_sentences = splitter.split(tweet)
pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
for sentence in pos_tagged_sentences:
    for words in sentence:
        print(words)
    print("\n")

#### POS tagging using spacy

Refer to this link for more explanation and examples- https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/

In [51]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(tweet)

for token in doc:
    print(f'{token.text:{10}} {token.pos_:{10}} {token.tag_:{10}}')

#### Comparison of nltk and spacy POS tagging results

Does one tagger perform better than the other? What are some of the differences you observe?
You can also try out tweets with more irregular text, which is likely to yield poorer results.

## Part 2: Syntactic analysis

### Chunking

####  Chunking with nltk

We can use regular expressions in NLTK for chunking. The following is a simple noun phrase chunking example. Study the `np` regular expression and try to interpret what it does. 

In [52]:
import nltk
# NLTK comes with RegexParser() function, which can help us with creating a simple noun phrase chunker.
np = ("NP: {<DT>?<JJ>*<NN>+}")

# create the regex for chunking
chunking = nltk.RegexpParser(np)

# tokenize the tweet sentence
sent_token = nltk.word_tokenize(tweet)

# POS tagging, a prerequisite for chunking
tagging = nltk.pos_tag(sent_token)
tagging

In [53]:
#!pip install svgling

# visualize the chunks
tree = chunking.parse(tagging)
print(tree)

In [54]:
# Print only the noun phrase chunks
for i in tree:
    if "NP" in str(i):
        print(i)

#### Chunking with spacy

To identify chunks in spacy, we need to first parse dependencies. 

For more details on spacy linguistic processing, refer to https://spacy.io/usage/linguistic-features. 

In [55]:
# displacy provides nice visualization features
from spacy import displacy

doc = nlp(tweet)    

# visualize sentence dependencies
displacy.render(doc, style='dep', jupyter = True, options = {'distance': 100})

In [56]:
# spacy provides more information about the chunks. 

df = pd.DataFrame(columns=["chunk.text", "chunk.root.text", "chunk.root.dep", "chunk.root.head.text"])
i=0

for chunk in doc.noun_chunks:
    # chunk text
    df.at[i,"chunk.text"] = chunk.text
    # chunk head
    df.at[i,"chunk.root.text"] = chunk.root.text
    df.at[i,"chunk.root.dep"] = chunk.root.dep_
    df.at[i,"chunk.root.head.text"] = chunk.root.head.text
    i+=1

df.head()

### Constituency parsing

There is limited built-in functionality when it comes to constituency parsing in nltk and spacy, although they provide wrappers to other existing tools like CoreNLP library. 

Stanza is a state-of-the-art NLP pipeline that includes constituency parsing (in addition to other linguistic analyses) and has a similar feel to spacy. You need to install it first. If you're interested in getting started with Stanza, see the link: https://stanfordnlp.github.io/stanza/#getting-started.

## Part 3: Semantic analysis

### Named entity recognition 

Simple examples of named entity recognition using nltk and spacy

#### Named entity recognition using spacy

In [57]:
from collections import Counter
from pprint import pprint
import en_core_web_sm
nlp = en_core_web_sm.load()

tweet = "Mark Zuckerberg is one of the founders of Facebook, a company from the United States"
doc = nlp(tweet)
print(tweet)
pprint([(X.text, X.label_) for X in doc.ents])

#### Named entity recognition using nltk

In [58]:
import nltk
# Download necessary packages
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
 
# a function that performs named entity recognition and prints the results
def nltk_ner(text): 
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))):
        if hasattr(chunk, 'label'):
            print(chunk.label(), ' '.join(c[0] for c in chunk))
    return

print(tweet) 
nltk_ner(tweet)
print('\n')

Which is better? Try some other sentences before you come to a conclusion.

If you need an efficient solution for NER, spaCy is a good choice with its pre-trained NER models. However, if you require more customization for domain-specific tasks, NLTK might be a better fit (you can train your own NER models). Ultimately, the choice depends on your project's requirements and your familiarity with the libraries. You may even choose to use both libraries in different parts of your NER pipeline if it best suits your needs.

## Part 4: Ambiguity

Let's try out some examples that were mentioned in class or others of your own. 

### Examples of lexical ambiguity 

Let's take the following two sentences.

```
sentence1 = "The key broke in the lock."
sentence2 = "The key problem was not one of quality but of quantity."
```
Both sentences have the word "key" but with different POS and meaning. 

### POS tagging using nltk for the above sentences
Let's see how our nltk POS tagger tags these sentences.

In [59]:
sentence1 = "The key broke in the lock."
sentence2 = "The key problem was not one of quality but of quantity."

print(sentence1)
forpos= sentence1.strip()
splitted_sentences = splitter.split(forpos)
pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
print(pos_tagged_sentences)

print("\n")

print(sentence2)
forpos= sentence2.strip()
splitted_sentences = splitter.split(forpos)
pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
print(pos_tagged_sentences)

What do you think? Are the results as expected? 


### POS tagging using spacy for the above sentences

In [60]:
sentence1 = "The key broke in the lock."
sentence2 = "The key problem was not one of quality but of quantity."

print(sentence1)
doc1 = nlp(sentence1)  
for token in doc1:
    print(f'{token.text:{10}} {token.pos_:{10}} {token.tag_:{10}}')

print("\n")

print(sentence2)

doc2 = nlp(sentence2)  
for token in doc2:
    print(f'{token.text:{10}} {token.pos_:{10}} {token.tag_:{10}}')


How does spacy results compare to nltk? Does it handle the word "key" better? 

Try other examples of interest. For example, you may try "I made her duck". 
